# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355311498929                   -1.10    6.0    162ms
  2   -9.356807297909       -2.83       -1.43    1.0   84.7ms
  3   -9.357068656680       -3.58       -2.78    2.1    151ms
  4   -9.357119566851       -4.29       -3.03    6.9    497ms
  5   -9.357119924350       -6.45       -3.17    1.0   81.1ms
  6   -9.357120119854       -6.71       -3.32    2.2   95.6ms
  7   -9.357120240287       -6.92       -3.47    1.0   80.4ms
  8   -9.357120288974       -7.31       -3.62    1.0   78.6ms
  9   -9.357120315887       -7.57       -3.88    1.1   84.5ms
 10   -9.357120322142       -8.20       -4.18    4.4    122ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74753866457                   -0.97    6.1    493ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     7
│     1
│     6
│     1
│     5
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.77681536482       -1.53       -1.30    2.9    264ms
  3   -18.79227107068       -1.81       -2.16    5.9    338ms
  4   -18.79258960611       -3.50       -2.54    6.1    358ms
  5   -18.79261926648       -4.53       -3.09    1.8    243ms
  6   -18.79262366571       -5.36       -3.35    5.1    333ms
  7   -18.79262411690       -6.35       -3.88    1.9    214ms
  8   -18.79262417462       -7.24       -4.18    3.0    286ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     11
│      3
│     17
│      3
│     15
│     13
│     12
│      6
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53280187703                   -0.84   10.0    1.90s
  2   -37.53979957030       -2.16       -1.22    2.1    747ms
  3   -37.54163940869       -2.74       -2.54    6.5    1.13s
  4   -37.56452002310       -1.64       -1.87   16.9    2.29s
  5   -37.56449947206   +   -4.69       -1.86    5.4    1.00s
  6   -37.56488617540       -3.41       -2.20    1.5    831ms
  7   -37.56497534660       -4.05       -2.71    1.4    770ms
  8   -37.56498029564       -5.31       -2.86    1.1    701ms
  9   -37.56498479799       -5.35       -3.64    2.5    1.11s
 10   -37.56498516092       -6.44       -3.93    6.8    1.44s
 11   -37.56498514674   +   -7.85    

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355174139605                   -1.10    6.2    175ms
  2   -9.356827140667       -2.78       -1.90    1.0   75.7ms
  3   -9.357096734159       -3.57       -2.60    4.6    135ms
  4   -9.357093779244   +   -5.53       -2.54    4.4    144ms
  5   -9.357119803367       -4.58       -3.46    1.0   70.2ms
  6   -9.357120186052       -6.42       -3.75    4.0    129ms
  7   -9.357120316576       -6.88       -4.38    1.5   81.8ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55024264978                   -0.84    9.9    1.78s
  2   -37.54958803944   +   -3.18       -1.60    2.6    676ms
  3   -26.69152120835   +    1.04       -0.55   11.0    2.01s
  4   -37.50985456795        1.03       -1.57    9.0    1.59s
  5   -37.52983775629       -1.70       -1.74    1.8    729ms
  6   -37.54983768467       -1.70       -1.92    1.2    656ms
  7   -37.56185898564       -1.92       -2.24    2.9    907ms
  8   -37.36541246098   +   -0.71       -1.41    6.5    1.62s
  9   -37.55921547746       -0.71       -2.02    5.2    1.17s
 10   -37.56493157047       -2.24       -2.77    2.6    953ms
 11   -37.56497933795       -4.32       -3.07    3.8    1.10s
 12   -37.56498245694       -5.51       -3.39    3.8    918ms
 13   -37.56498320472       -6.13       -3.57    2.8    1.28s
 14   -37.56498470274       -5.82       -3.84    2.1    720ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.